In [1]:
import sympy as sp
from lbmpy.stencils import LBStencil, Stencil
from lbmpy.moments import MOMENT_SYMBOLS, moment_matrix

In [2]:
d3q19 = LBStencil(Stencil.D3Q19)

In [3]:
x, y, z = MOMENT_SYMBOLS
one = sp.core.sympify(1)
c2 = x**2+y**2+z**2
c4 = c2**2

moments = [ 
    one,
    x,
    y,
    z,
    c2-one,
    3*x**2-c2,
    y**2-z**2,
    x*y,
    y*z,
    z*x,
    (3*c2-5)*x,
    (3*c2-5)*y,
    (3*c2-5)*z,
    (y**2-z**2)*x,
    (z**2-x**2)*y,
    (x**2-y**2)*z,
    3*c4-6*c2+one,
    (2*c2-3)*(3*x**2-c2),
    (2*c2-3)*(y**2-z**2)
]

M = moment_matrix(moments, stencil=d3q19)

In [4]:
five36 = sp.Rational(5,36)
one9 = sp.Rational(1,9)
one12 = sp.Rational(1,12)
one36 = sp.Rational(1,36)
one72 = sp.Rational(1,72)

cs = sp.sqrt(sp.Rational(1,3))

w = [ sp.Rational(1,3) ] + [ sp.Rational(1,18) ]*6 + [ sp.Rational(1,36) ]*12

ww =  [ [[0]*3]*3 ] \
    + [ [ [ -one9, 0, 0], [0, five36, 0], [0, 0,  -one9] ] ]*2 \
    + [ [ [five36, 0, 0], [0,  -one9, 0], [0, 0,  -one9] ] ]*2 \
    + [ [ [ -one9, 0, 0], [0,  -one9, 0], [0, 0, five36] ] ]*2 \
    + [ [ [-one72, -one12, 0], [ -one12, -one72, 0], [ 0, 0, one36] ],
        [ [-one72,  one12, 0], [  one12, -one72, 0], [ 0, 0, one36] ],
        [ [-one72,  one12, 0], [  one12, -one72, 0], [ 0, 0, one36] ],
        [ [-one72, -one12, 0], [ -one12, -one72, 0], [ 0, 0, one36] ],
        [ [ one36, 0, 0], [ 0, -one72,  one12], [ 0,  one12, -one72] ],
        [ [ one36, 0, 0], [ 0, -one72, -one12], [ 0, -one12, -one72] ],
        [ [-one72, 0, -one12], [ 0, one36, 0], [ -one12, 0, -one72] ],
        [ [-one72, 0,  one12], [ 0, one36, 0], [  one12, 0, -one72] ],
        [ [ one36, 0, 0], [ 0, -one72, -one12], [ 0, -one12, -one72] ],
        [ [ one36, 0, 0], [ 0, -one72,  one12], [ 0,  one12, -one72] ],
        [ [-one72, 0,  one12], [ 0, one36, 0], [  one12, 0, -one72] ],
        [ [-one72, 0, -one12], [ 0, one36, 0], [ -one12, 0, -one72] ] ]

In [5]:
x, y, z = sp.symbols(['x','y','z'])
k = sp.symbols('kappa')
Gamma = sp.symbols('Gamma')
rho = sp.symbols('rho',cls=sp.Function)(x,y,z)
phi = sp.symbols('phi',cls=sp.Function)(x,y,z)
pb = sp.symbols('p_b')
mu = sp.symbols('mu')
u = sp.Matrix(sp.symbols('u_0, u_1, u_2'))
D2rho = sum([ sp.Derivative(rho,a,2) for a in [x,y,z] ])
D2phi = sum([ sp.Derivative(phi,a,2) for a in [x,y,z] ])
Drho2 = sp.Matrix([ [ sp.Derivative(rho,a)*sp.Derivative(rho,b) for b in [x,y,z] ] for a in [x,y,z] ])
Dphi2 = sp.Matrix([ [ sp.Derivative(phi,a)*sp.Derivative(phi,b) for b in [x,y,z] ] for a in [x,y,z] ])
G = k*Drho2+k*Dphi2

c = sp.Matrix(d3q19)

def f(i):
    f = w[i]/cs**2*pb \
        + w[i]/cs**2*rho*c[i,:].dot(u) \
        + w[i]/(2*cs**4)*rho*(u.T@(c[i,:].T@c[i,:]-cs**2*sp.eye(3))@u)[0] \
        - w[i]/cs**2*(k*rho*D2rho+k*phi*D2phi) \
        + sum([ ww[i][j][j]/cs**2*G[j,j] for j in range(3) ]) \
        + sum([ ww[i][j][(j+1)%3]/cs**2*G[j,(j+1)%3] for j in range(3) ])
    return f

feq = [ f(i) for i in range(0,19) ]
feq[0] = rho - sum(feq[1:])

def g(i):
    g = w[i]/cs**2*Gamma*mu \
        + w[i]/cs**2*phi*c[i,:].dot(u) \
        + w[i]/(2*cs**4)*phi*u.dot((c[i,:].T@c[i,:]-cs**2*sp.eye(3))@u)
    return g

geq = [ g(i) for i in range(0,19) ]
geq[0] = phi - sum(geq[1:])

In [6]:
mf = M@sp.Matrix(feq)
sp.simplify(mf)

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 rho(x, y, z)],
[                                                                                                                                                                                                                                                                                                                             

In [7]:
mg = M@sp.Matrix(geq)
sp.simplify(mg)

Matrix([
[                                                                               phi(x, y, z)],
[                                                                           u_0*phi(x, y, z)],
[                                                                           u_1*phi(x, y, z)],
[                                                                           u_2*phi(x, y, z)],
[3*Gamma*mu + u_0**2*phi(x, y, z) + u_1**2*phi(x, y, z) + u_2**2*phi(x, y, z) - phi(x, y, z)],
[                                                  (2*u_0**2 - u_1**2 - u_2**2)*phi(x, y, z)],
[                                                             (u_1**2 - u_2**2)*phi(x, y, z)],
[                                                                       u_0*u_1*phi(x, y, z)],
[                                                                       u_1*u_2*phi(x, y, z)],
[                                                                       u_0*u_2*phi(x, y, z)],
[                                        